# Creating the Funding Transaction

In this section, we’ll create a Lightning channel funding transaction from scratch using Python. We’ll walk through each part of the transaction—how it’s constructed, signed, and how peers exchange messages to share the necessary information to make it happen. We'll test everything using Bitcoin Core in regtest mode.


## Setup

For this exercise we'll need Bitcoin Core. This notebook has been tested with v29.0

Below, set the paths for:

    The bitcoin core functional test framework directory.
    The directory containing taproot-lightning-channels-workshop.

You'll need to edit these next two lines for your local setup.


In [1]:
path_to_bitcoin_functional_test = "/home/pins-dev/Projects/bitcoin/build/test/functional"
path_to_taproot_workshop = "/home/pins-dev/Projects/Taproot-Lightning-Channels-Workshop"


### Setup bitcoin core test framework

Start up regtest mode, delete any regtest network history so we are starting from scratch.

In [ ]:
import sys

# Add the functional test framework to our PATH
sys.path.insert(0, path_to_bitcoin_functional_test)
from test_framework.test_shell import TestShell

# Add the bitcoin-tx-tutorial functions to our PATH
sys.path.insert(0, path_to_taproot_workshop)
from functions import *

import json

# Setup our regtest environment
test = TestShell().setup(
    num_nodes=1, 
    setup_clean_chain=True
)

node = test.nodes[0]

# Create a new wallet and address to send mining rewards so we can fund our transactions
node.createwallet(wallet_name='mywallet')
address = node.getnewaddress()

# Generate 101 blocks so that the first block subsidy reaches maturity
result = node.generatetoaddress(nblocks=101, address=address, called_by_framework=True)

# Check that we were able to mine 101 blocks
assert(node.getblockcount() == 101)

# Create a new address to alice so she can sign the funding transaction
alice_wallet_ctx = generate_taproot_ctx()
alice_funding_address = generate_taproot_address(alice_wallet_ctx, index=0, change=False)
print(f"Alice funding address created: {alice_funding_address}")

# Create a new address to bob so he can sign the funding transaction
bob_wallet_ctx = generate_taproot_ctx()
bob_funding_address = generate_taproot_address(bob_wallet_ctx, index=0, change=False)
print(f"Bob funding address created: {bob_funding_address}")

# Envia 1 BTC para Alice e Bob
txid = node.sendmany("", {alice_funding_address: 1, bob_funding_address: 1})
result = node.generatetoaddress(nblocks=1, address=address, called_by_framework=True)

2025-08-17T17:57:14.366000Z TestFramework (INFO): PRNG seed is: 9172606118396639736
2025-08-17T17:57:14.368000Z TestFramework (INFO): Initializing test directory /tmp/bitcoin_func_test_evwwtr7z
Alice funding address created: bcrt1pjvfyu8ldh6ws7aqkxafgp20pzpsddjhylv04tdvarca7xapfmssse9vvu2
Bob funding address created: bcrt1pa37w0nf7pk7t47zfv8tgyecmm42rtgm0g268fdea3l5atcdysfxq8ad7ml
